# Задача 1.

Задача – написать функцию в python, которая будет считать retention игроков (по дням от даты регистрации игрока). Данные лежат в папке shared и имеют следующую структуру:

shared/problem1-reg_data.csv – данные о времени регистрации
reg_ts	    uid
906166566	2
906344325	3
906686169	4
906893386	5
906980227	6

shared/problem1-auth_data.csv – данные о времени захода пользователей в игру
auth_ts	    uid
906166566	2
924422172	2
937374732	2
947425117	2
955630339	2

In [1]:
#Импортируем библиотеки
import pandas as pd
import numpy as np
import scipy.stats as ss

In [2]:
#Считываем данные из предоставленных нам данных
reg_data = pd.read_csv('~/shared/problem1-reg_data.csv', sep = ';')
auth_data = pd.read_csv('~/shared/problem1-auth_data.csv', sep = ';')

In [3]:
#Приводим даты к корректному виду
reg_data['reg_ts'] = pd.to_datetime(reg_data['reg_ts'], unit='s')
auth_data['auth_ts'] = pd.to_datetime(auth_data['auth_ts'], unit = 's')

In [34]:
reg_data.head()

,reg_ts,uid
0,1998-11-18 09:43:43,1
1,1999-07-22 22:38:09,2
2,2000-01-13 22:27:27,3
3,2000-05-28 14:19:01,4
4,2000-09-16 11:21:53,5


In [33]:
auth_data.head()

,auth_ts,uid
0,1998-11-18 09:43:43,1
1,1999-07-22 22:38:09,2
2,1999-07-25 16:46:46,2
3,1999-07-31 03:50:15,2
4,1999-08-05 17:49:39,2


In [6]:
#Проверяем данные на наличие пропусков:
reg_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   reg_ts  1000000 non-null  datetime64[ns]
 1   uid     1000000 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 15.3 MB


In [7]:
auth_data.info()
auth_data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9601013 entries, 0 to 9601012
Data columns (total 2 columns):
 #   Column   Dtype         
---  ------   -----         
 0   auth_ts  datetime64[ns]
 1   uid      int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 146.5 MB


auth_ts    0
uid        0
dtype: int64

Пропуски отсутствуют.

In [8]:
#Проверяем данные на наличие дубликатов:
reg_data.nunique()

reg_ts    1000000
uid       1000000
dtype: int64

In [9]:
auth_data.nunique()

auth_ts    9180915
uid        1000000
dtype: int64

Исходя из EDA: 
- в датасете reg_data каждому значению столбца uid соответствует единственное значение столбца reg_ts, 
- в датасете auth_data одному и тому же значению столбца uid может соответствовать несколько значений столбца auth_ts.

In [10]:
# Определим период для анализа retention. Посмотрим границы временного интервала данных
print(reg_data.reg_ts.min())
print(reg_data.reg_ts.max())
print(auth_data.auth_ts.min())
print(auth_data.auth_ts.max())

1998-11-18 09:43:43
2020-09-23 15:17:24
1998-11-18 09:43:43
2020-09-23 15:17:24


In [11]:
# Проанализируем retention за первые 2 недели января 2020, чтобы данные читаемо отображались в результате
start_analyse_period = pd.to_datetime('2020-01-01')
end_analyse_period = pd.to_datetime('2020-01-15')

In [12]:
#Пишем функцию для расчёта retention
def calculate_retention(reg_data, auth_data, period = 'D'):
    '''
    Рассчитывает retention по дням за заданный период времени.
        Параметры: 
            reg_data: данные о данные о времени регистрации
            auth_data: данные о времени захода пользователей в игру
        
        Возвращаемое значение:
            retention по дням за анализируемый период
    '''
    
    #Объединяем датафреймы для расчёта retention
    data = reg_data.merge(auth_data, how = 'left', on = 'uid')
    
    #Фильтруем по периоду времени
    data = data[(data['reg_ts'] >= start_analyse_period) & (data['reg_ts'] <= end_analyse_period)]
    data = data[(data['auth_ts'] >= start_analyse_period) & (data['auth_ts'] <= end_analyse_period)]
    
    #Извлекаем дни регистрации и захода в игру в анализируемом периоде
    data['registration_day'] = data['reg_ts'].dt.floor(period)
    data['authorisation_day'] = data['auth_ts'].dt.floor(period)
    
    #Рассчитываем номер периода в днях со дня регистрации
    data['day_play'] = (data['authorisation_day'] - data['registration_day']).dt.days
    
    #Считаем количество пользователей, зарегистрировавшихся в периоде
    count_registrations = data.groupby('registration_day')['uid'].nunique()
    
    #Считаем количество вернувшихся пользователей
    retention = data.groupby(['registration_day', 'day_play'])['uid'].nunique().unstack(fill_value = 0)
    
    #Считаем retention rate
    retention_rate = retention.div(count_registrations, axis = 0) * 100
    
    return retention_rate

#Применение функции
retention_rate = calculate_retention(reg_data, auth_data)
retention_rate

day_play,0,1,2,3,4,5,6,7,8,9,10,11,12,13
registration_day,,,,,,,,,,,,,,
2020-01-01,100.0,2.639020,4.524034,3.393025,5.655042,5.843544,7.540057,4.995287,4.429783,4.429783,5.372290,6.503299,4.618285,4.901037
2020-01-02,100.0,2.351834,4.515522,5.174036,4.797742,5.832549,7.714017,5.926623,5.644403,5.268109,5.644403,5.644403,4.515522,0.000000
2020-01-03,100.0,2.347418,3.380282,4.413146,4.507042,6.854460,7.511737,5.446009,4.507042,4.225352,5.258216,5.821596,0.000000,0.000000
2020-01-04,100.0,1.499531,4.498594,5.060918,5.342081,5.716963,7.029053,5.435801,4.217432,4.873477,4.779756,0.000000,0.000000,0.000000
2020-01-05,100.0,1.964453,4.490178,4.583723,4.396632,4.770814,7.857811,5.238541,3.928906,5.425631,0.000000,0.000000,0.000000,0.000000
2020-01-06,100.0,2.897196,4.485981,4.953271,3.831776,4.672897,6.635514,5.887850,5.233645,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-07,100.0,1.865672,4.850746,4.384328,6.436567,5.037313,8.488806,5.130597,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-08,100.0,2.234637,4.003724,4.283054,5.121043,5.027933,8.472998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-09,100.0,1.858736,3.996283,5.483271,4.460967,6.040892,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Задача 2.

Имеются результаты A/B теста, в котором двум группам пользователей предлагались различные наборы акционных предложений. 
Известно, что ARPU в тестовой группе выше на 5%, чем в контрольной. 
При этом в контрольной группе 1928 игроков из 202103 оказались платящими, а в тестовой – 1805 из 202667.

Какой набор предложений можно считать лучшим? Какие метрики необходимы для принятия решения?

In [13]:
#Подгружаем данные
import requests
from urllib.parse import urlencode

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/d/SOkIsD5A8xlI7Q'
total_url = base_url + urlencode(dict(public_key = public_key))
response = requests.get(total_url)
download_url = response.json()['href']

In [14]:
#Считываем данные
task_2 = pd.read_csv(download_url, sep = ';')
task_2

,user_id,revenue,testgroup
0,1,0,b
1,2,0,a
2,3,0,a
3,4,0,b
4,5,0,b
...,...,...,...
404765,404766,0,a
404766,404767,0,b
404767,404768,231,a
404768,404769,0,a


In [15]:
#Проверяем данные:
task_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404770 entries, 0 to 404769
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   user_id    404770 non-null  int64 
 1   revenue    404770 non-null  int64 
 2   testgroup  404770 non-null  object
dtypes: int64(2), object(1)
memory usage: 9.3+ MB


Пропуски отсутствуют.

In [16]:
task_2.nunique()

user_id      404770
revenue        1477
testgroup         2
dtype: int64

In [17]:
#Проверяем нашу теорию, что ARPU в тестовой группе выше на 5%, чем в контрольной
ARPU_testgroup_b = task_2.query('testgroup == "b"').revenue.sum() / task_2.query('testgroup == "b"').user_id.count()
print(ARPU_testgroup_b)
ARPU_testgroup_a = task_2.query('testgroup == "a"').revenue.sum() / task_2.query('testgroup == "a"').user_id.count()
print(ARPU_testgroup_a)
ARPU_testgroup_b / ARPU_testgroup_a * 100

26.75128659327863
25.413719736965806


105.2631683600699

In [18]:
#Проверяем количество платящих и общее число участников групп
print(f'''Число участников тестовой группы: {task_2.query('testgroup == "b"').user_id.count()}''')
print(f'''Число платящих тестовой группы: {task_2.query('testgroup == "b" & revenue > 0').user_id.count()}''')
print(f'''Число участников контрольной группы: {task_2.query('testgroup == "a"').user_id.count()}''')
print(f'''Число участников контрольной группы: {task_2.query('testgroup == "a" & revenue > 0').user_id.count()}''')

Число участников тестовой группы: 202667
Число платящих тестовой группы: 1805
Число участников контрольной группы: 202103
Число участников контрольной группы: 1928


In [19]:
#Посчитаем конверсию в платящих по группам
converse_b = task_2.query('testgroup=="b" & revenue>0').user_id.count() / task_2.query('testgroup=="b"').user_id.count() * 100
converse_a = task_2.query('testgroup=="a" & revenue>0').user_id.count() / task_2.query('testgroup=="a"').user_id.count() * 100
print(f'Конверсия в платящего в тестовой группе: {converse_b}')
print(f'Конверсия в платящего в контрольной группе: {converse_a}')

Конверсия в платящего в тестовой группе: 0.8906235351586593
Конверсия в платящего в контрольной группе: 0.9539690157988748


In [20]:
#Посчитаем ARPPU в группах
ARPPU_b = task_2.query('testgroup == "b" & revenue > 0').revenue.sum() / task_2.query('testgroup == "b" & revenue > 0').user_id.count()
print(f'Средний чек платящего клиента в тестовой группе: {ARPPU_b}')
ARPPU_a = task_2.query('testgroup == "a" & revenue > 0').revenue.sum() / task_2.query('testgroup == "a" & revenue > 0').user_id.count()
print(f'Средний чек платящего клиента в контрольной группе: {ARPPU_a}')

Средний чек платящего клиента в тестовой группе: 3003.6581717451522
Средний чек платящего клиента в контрольной группе: 2663.9984439834025


Проверим статзначимость ARPU.

In [21]:
#Проверим нормальность распределения и дисперсии выручки
print(ss.shapiro(task_2.query('testgroup == "b"').revenue))
print(ss.shapiro(task_2.query('testgroup == "a"').revenue))

ShapiroResult(statistic=0.06588172912597656, pvalue=0.0)
ShapiroResult(statistic=0.008876502513885498, pvalue=0.0)


/opt/tljh/user/lib/python3.7/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [22]:
ss.levene(task_2.query('testgroup == "b"').revenue, task_2.query('testgroup == "a"').revenue)

LeveneResult(statistic=0.3896289474701388, pvalue=0.5324948591043842)

Распределения не нормальны, дисперсии не имеют существенных различий. С учётом большого объёма выборки, можно пренебречь ненормальностью распределения и использовать t-тест.

In [23]:
#Проверим статзначимость ARPU
ss.ttest_ind(task_2.query('testgroup == "b"').revenue, task_2.query('testgroup == "a"').revenue)

Ttest_indResult(statistic=0.6242026493616787, pvalue=0.532494858971837)

Показатель ARPU не оказывает статзначимого влияния на исследование. Несмотря на 5% превышение среднего чека в контрольной группе в сравнении с тестовой, данное обстоятельство можно не учитывать.

Проверим статзначимость ARPPU.

In [35]:
#Проверим статзначимость ARPPU. 
#Параметры выручки остаются неизменными, поэтому проведём ещё один t-тест, но уже среди платящих пользователей 
ss.ttest_ind(task_2.query('testgroup == "b" & revenue > 0').revenue, task_2.query('testgroup == "a" & revenue > 0').revenue)

Ttest_indResult(statistic=1.5917100176862002, pvalue=0.11153459157259504)

Показатель ARPPU также не оказывает статзначимого влияния на исследование.

Проверим статзначимость конверсии в платящего пользователя.

In [25]:
#Для расчёта статзначимости конверсии преобразуем исходный датафрейм с добавлением необходимых столбцов
task_2['conversion'] = task_2.revenue.apply(lambda x: 1 if x > 0 else 0)
task_2_converse = task_2.groupby('testgroup', as_index = False) \
                        .agg(total_users = ('user_id', 'count'), paying_users = ('conversion', 'sum'))
task_2_converse

,testgroup,total_users,paying_users
0,a,202103,1928
1,b,202667,1805


In [36]:
#Проверяем статзначимость конверсии в платящего пользователя обеих групп 
from statsmodels.stats.proportion import proportions_ztest
z_stat, p_value_conv = proportions_ztest(task_2_converse.paying_users, task_2_converse.total_users)
z_stat, p_value_conv

(2.108028495889841, 0.035028524642854865)

Конверсия в платящего пользователя статзначима в нашем исследовании. Несмотря на 5% преимущество по среднему чеку, определяющее значение в нашем исследовании имеет показатель конверсии в платящего пользователя. Посмотрим на общую информацию по выручке в разрезе групп.

In [31]:
task_2[(task_2['revenue'] > 0) & (task_2['testgroup'] == "a")].revenue.describe()

count     1928.000000
mean      2663.998444
std       9049.039763
min        200.000000
25%        257.000000
50%        311.000000
75%        361.000000
max      37433.000000
Name: revenue, dtype: float64

In [32]:
task_2[(task_2['revenue'] > 0) & (task_2['testgroup'] == "b")].revenue.describe()

count    1805.000000
mean     3003.658172
std       572.619709
min      2000.000000
25%      2513.000000
50%      3022.000000
75%      3478.000000
max      4000.000000
Name: revenue, dtype: float64

По итогам исследования можно сделать вывод, что набор предложений в тестовой группе не является превосходящим по качеству предложения контрольной. Необходимо принять во внимание конверсию в платящего пользователя контрольной группы, которая показывает, что клиенты охотнее делают выбор в пользу акционных предложений именно этой группы. Больший средний чек у тестовой группы, судя по предварительному анализу, обусловен либо более дорогой стоимостью предложений, либо большим доходом участников тестовой группы. Так как максимальное значение выручки от одного клиента в тестовой группе значительно превосходит и среднее по группе, и значение 75-% перцентиля. Скорее всего, на ARPU оказал влияние выброс, исказив показатель.

# Задача 3.

В исследуемом приложении каждый месяц проводятся тематические события, ограниченные по времени. В них игроки могут получить уникальные предметы, дополнительные монеты или бонусы. Для получения награды требуется пройти ряд уровней за определенное время. С помощью каких метрик можно оценить результаты последнего прошедшего события?

Предположим, в другом событии мы усложнили механику событий так, что при каждой неудачной попытке выполнения уровня игрок будет откатываться на несколько уровней назад. Изменится ли набор метрик оценки результата? Если да, то как?

Метрики для анализа события:
1. DAU. При сравнении численности аудитории, чтобы проанализировать рост или спад интереса к событию в сравнении с предыдущим наиболее подойдёт DAU, на которой мы увидим количество пользователей в игре в день события.
2. Конверсия в участника. Данная метрика покажет, какая доля пользователей из общего числа в этот день решила поучаствовать в событии. Таким образом, мы выявим интерес к событию среди лояльных пользователей.
3. Процент успешно прошедших дополнительные уровни пользователей от общего числа участников. Сравнение данной метрики с аналогичной в предыдущем событии покажет не были ли уровни слишком лёгкие или сложные для прохождения в целях анализа при планировании следующего мероприятия.

При усложнении прохождения уровней в мероприятии набор метрик изменится следующим образом:
Метрики с предыдущего этапа останутся, так как они нам необходимы для регулярного мониторинга.
При этом я добавляю следующие метрики:
1. Длительность нахождения пользователя в игре, чтобы посмотреть, не оттолкнёт ли усложнение уровней желание участников пройти мероприятие.
2. Retention следующих мероприятий. Не снизит ли усложнившаяся механика количество участников следующих событий после возможного негативного опыта с мероприятием с изменившейся сложностью. Если retention начнёт снижаться, стоит задуматься об облегчении уровней в целях удержания аудитории.